## 4 Model Training

In [ ]:
import pandas as pd
from happytransformer import HappyTextToText, TTSettings

In [ ]:
happy_tt = HappyTextToText("text2text-generation", "pszemraj/grammar-synthesis-small")

args = TTTrainArgs(batch_size=8)
happy_tt.train("/content/drive/MyDrive/corrupt/train_3k.csv", args=args)
happy_tt.model.save_pretrained("french_model")

### Load saved model

In [3]:
happy_tt = HappyTextToText("French", "./french_model")

args = TTSettings(num_beams=5, min_length=1, max_length=200)

sentence = 'I are happy'
result = happy_tt.generate_text("gec: " + sentence, args=args)
result.text

11/08/2023 15:35:27 - INFO - happytransformer.happy_transformer -   Using device: mps
11/08/2023 15:35:27 - INFO - happytransformer.happy_transformer -   Moving model to mps
11/08/2023 15:35:27 - INFO - happytransformer.happy_transformer -   Initializing a pipeline


'I am happy.'